# QUANTUM ESPRESSO Example App

**Author: Aliaksandr Yakutovich (THEOS, EPFL)**

This automatic workflow allows to optimize geometry, compute the band structure of a material with a minimal set of input parameters.

It is powered by:
- [QUANTUM ESPRESSO](https://www.quantum-espresso.org/) as the quantum engine
- [AiiDA](http://www.aiida.net) as the automation platform
- [AiiDA-QUANTUMESPRESSO](https://github.com/aiidateam/aiida-quantumespresso) as the curated pseudopotential family
- Custom-made workflows for AiiDA to manage the selection of parameters, the error handling, ...
- [AppMode for Jupyter](http://github.com/oschuett/jupyter_appmode) to create a simple UI

### Example steps to run:
1. SCF run
1. Relaxation run
1. Calculate band structure

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import ase.io
import nglview
import tempfile
import ipywidgets as ipw
from os import path

from IPython.display import FileLink

from fileupload import FileUploadWidget
from aiidalab_widgets_base import CodeDropdown
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.work.run import submit, run
from aiida.orm.utils import CalculationFactory
from aiida.orm.data.upf import get_pseudos_from_structure
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.parameter import ParameterData
from aiida.orm import load_node

from time import sleep 

Calculation = CalculationFactory('quantumespresso.pw')

In [ ]:
arrow_down = ipw.HTML("""<hr> <br /> <center><i class="fa fa-arrow-down" style="color:#B0B0B0;font-size:12em;" ></i></center>""")
hr = ipw.HTML('<hr>')
run_btn = ipw.Button(description='Submit calculation')

code_group = CodeDropdown(input_plugin='quantumespresso.pw', text="Select the code")

number_of_nodes = ipw.IntText(value=1, step=1,description = "that will be run on", disabled=False, layout=ipw.Layout(width="180px"),
    style={"description_width":"120px"},)

In [ ]:
def get_example_structure(key):
    from ase.io import read
    return read('structures/' + key)

class StructureUploadWidget(ipw.VBox):
    
    def __init__(self, text="Upload Structure", **kwargs):
        """ Upload a structure and 
        it in AiiDA database.

        :param text: Text to display before upload button
        :type text: str
        """

        self.file_upload = FileUploadWidget(text)
        structures = {
                "Select structure": False,
                }
        self.structure_select = ipw.Dropdown(
                options=[],
                description='Or choose from examples:',
                style={'description_width': '160px'},
                disabled=False)
        self.viewer = nglview.NGLWidget()
        self.structure_description = ipw.Text(
            placeholder="Description (optional)")

        self.structure_ase = None
        supported_formats = ipw.HTML("""All supported structure formats are listed <a href="https://wiki.fysik.dtu.dk/ase/_modules/ase/io/formats.html" target="_blank">here</a>""")
        select = ipw.VBox([ipw.HBox([self.file_upload, self.structure_select]),
                           supported_formats])
                           
        children = [select, self.viewer, self.structure_description]

        super(StructureUploadWidget, self).__init__(
            children=children, **kwargs)

        self.file_upload.observe(self._on_file_upload, names='data')
        self.structure_select.observe(self._on_structure_select, names=['value'])
        structures = {
            "Select structure": False,
            'Silicon' : get_example_structure('si.cif'),
            'Gallium arsenide': get_example_structure('gaas.cif'),
        }
        self.structure_select.options = structures
        self.structure_select.value = False


    # pylint: disable=unused-argument
    def _on_file_upload(self, change):
        self.tmp_folder = tempfile.mkdtemp()
        tmp = self.tmp_folder + '/' + self.file_upload.filename
        with open(tmp, 'w') as f:
            f.write(self.file_upload.data)
        structure_ase = self.get_ase(self.tmp_folder + '/' + self.file_upload.filename)
        self.select_structure(s=structure_ase, name=self.file_upload.filename)

    def _on_structure_select(self, change):
        global atoms
        indx = change['owner'].index
        atoms = change['new']
        if atoms is False:
            self.select_structure(s=None, name=None)
            return None
        formula = atoms.get_chemical_formula()
        self.select_structure(s=atoms, name=formula)

    def select_structure(self, s, name):
        if s is None:
            self.structure_ase = None
            self.structure_description.value = ""
            submit_out.layout.visibility = 'hidden'
            self.refresh_view()
            return
        submit_out.layout.visibility = 'visible'
        self.structure_description.value = "{} ({})".format(s.get_chemical_formula(), name)
        self.structure_ase = s
        self.refresh_view()

    def get_ase(self, fname):
        try:
            traj = ase.io.read(fname, index=":")
        except AttributeError:
            print("Looks like {} file does not contain structure coordinates".
                  format(fname))
            return None
        if len(traj) > 1:
            print(
                "Warning: Uploaded file {} contained more than one structure. I take the first one."
                .format(fname))
        return traj[0]

    def refresh_view(self):
        viewer = self.viewer
        # Note: viewer.clear() only removes the 1st component
        # pylint: disable=protected-access
        for comp_id in viewer._ngl_component_ids:
            viewer.remove_component(comp_id)

        if self.structure_ase is None:
            return

        viewer.add_component(nglview.ASEStructure(
            self.structure_ase))  # adds ball+stick
        viewer.add_unitcell()

    @property
    def structure(self):
        if self.structure_ase is None:
            return None
        else:
            from aiida.orm.data.structure import StructureData
            return StructureData(ase=self.structure_ase)


In [ ]:
def setup_calc():
    options =  {
        'max_wallclock_seconds': 3600*2,
        'resources':{
            'num_machines': number_of_nodes.value,
        }
    }
    kpoints = KpointsData()
    kpoints.set_kpoints_mesh([kpx.value, kpy.value, kpz.value])

    if code_group.selected_code is None:
        print ("Please select a code")
        return None

    if structure_widget.structure is None:
        #print ("Please select a structure")
        return None

    parameters = {
        'CONTROL': {
            'calculation': run_type.value,
            'restart_mode': 'from_scratch',
            'wf_collect': True,
        },
        'SYSTEM': {
            'ecutwfc': 50.,
            'ecutrho': 200.,
        },
        'ELECTRONS': {
            'scf_must_converge': False,
            'conv_thr': 1.e-6,
        },
    }


    if not 'structure_global' in globals():
        global structure_global
        structure_global = structure_widget.structure

    settings = {}
    if run_type.value == "bands":
        settings['also_bands'] = True # instruction for output parser

    inputs = {
        'code': code_group.selected_code,
        'structure': structure_global,
        'pseudo': get_pseudos_from_structure(structure_global, pseudo_family.value),
        'kpoints': kpoints,
        'settings': ParameterData(dict=settings),
        '_options': options,
    }
    
    if 'parent_folder' in globals() and restart_from_previous.value:
        inputs['parent_folder'] = parent_folder
        if run_type.value != "bands":
            parameters['CONTROL']['restart_mode'] = 'restart'
    
    inputs['parameters'] = ParameterData(dict=parameters)

    return inputs

In [ ]:
class ProgressBar(ipw.VBox):
    def __init__(self, state='NEW', **kwargs):
        self.correspondance = {
            "NEW" : 0,
            "TOSUBMIT": 1,
            "SUBMITTING": 2,
            "WITHSCHEDULER": 3,
            "COMPUTED": 4,
            "RETRIEVING": 5,
            "PARSING": 6,
            "FINISHED": 7,
            "FAILED": 7,
            "SUBMISSIONFAILED": 7,
            "RETRIEVALFAILED": 7,
            "PARSINGFAILED": 7,
        }
        self.bar = ipw.IntProgress(
            value=0,
            min=0,
            max=7,
            step=1,
            description='Progress:',
            bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
            orientation='horizontal',
            layout=ipw.Layout(width="800px")
        )
        self.state = ipw.HTML(description="Calculation state:", value=state, style = {'description_width': 'initial'}) 
        children = [self.bar, self.state]
        super(ProgressBar, self).__init__(children=children, **kwargs)
    
    def update_state(self, state, scheduler_state):
        self.bar.value = self.correspondance[state]
        if state == "WITHSCHEDULER" and scheduler_state is not None:
            self.state.value = "{} ({})".format(state, scheduler_state)
        else:
            self.state.value = state
        if state == 'FINISHED':
            self.bar.bar_style = 'success'
        elif state in ["FAILED", "SUBMISSIONFAILED", "RETRIEVALFAILED", "PARSINGFAILED"]:
            self.bar.bar_style = 'danger'
        else:
            self.bar.bar_style = 'info'
            

In [ ]:
def submit_calculation(b):
    global calculation
    inputs = setup_calc()
    
    if inputs is None:
        pass
    else:
        calc = submit(Calculation.process(), **inputs)
        calculation = load_node(calc.pid)
        progress = ProgressBar(state=calculation.get_state())
        display(progress)
        while not calculation.is_sealed:
            progress.update_state(calculation.get_state(), calculation.get_scheduler_state())
            sleep(0.1)
        progress.update_state(calculation.get_state(), calculation.get_scheduler_state())
        if hasattr(calculation.out, 'output_structure'):
            optimized_structure = calculation.out.output_structure
        global parent_folder
        parent_folder = calculation.out.remote_folder
        display(arrow_down)
        show_results()

In [ ]:
def calculation_settings():
    global run_type, kpx, kpy, kpz, restart_from_previous, pseudo_family
    run_type = ipw.ToggleButtons(options=['scf', 'relax', 'vc-relax', 'bands'],
                             description='Calculation type:',
                             style = {'description_width': 'initial'},
                            )
    kpx = ipw.BoundedIntText(value=2, min=1, step=1, description='k-points', layout=ipw.Layout(width="144px"))
    kpy = ipw.BoundedIntText(value=2, min=1, step=1, description='', layout=ipw.Layout(width="50px"))
    kpz = ipw.BoundedIntText(value=2, min=1, step=1, description='', layout=ipw.Layout(width="50px"))
    kpoints = ipw.HBox([kpx, kpy, kpz])
    restart_from_previous = ipw.Checkbox(
        description='Restart from previous calculation',
        style = {'description_width': 'initial'}
    )
    pseudo_family = ipw.ToggleButtons(
        options = {
            'SSSP efficiency': 'SSSP_efficiency_v1.0',
            'SSSP accuracy': 'SSSP_accuracy_v1.0',
        },
        description='Pseudopotential family:',
        style = {'description_width': 'initial'},
    )
    if 'parent_folder' in globals():
        restart_from_previous.disabled = False
        restart_from_previous.value = True
    else:
        restart_from_previous.disabled = True
        restart_from_previous.value = False
    specify_computer_code = ipw.HTML("""<a href="https://dev-aiidalab.materialscloud.org/user/aliaksandr.yakutovich@epfl.ch/apps/apps/aiidalab-widgets-base/setup_computer.ipynb" target="_blank"><button>Setup another Computer</button></a> <a href="https://dev-aiidalab.materialscloud.org/user/aliaksandr.yakutovich@epfl.ch/apps/apps/aiidalab-widgets-base/setup_code.ipynb" target="_blank"><button>Setup another Code</button></a>""")

    display(arrow_down,
            ipw.HBox([code_group, number_of_nodes, ipw.HTML("node(s)")]),
            specify_computer_code,
            run_type,
            pseudo_family,
            kpoints,
            restart_from_previous,
            run_btn)

In [ ]:
def show_results():
    to_display = []
    if hasattr(calculation.out, 'retrieved'):
        from shutil import copyfile
        src_path = path.join(calculation.out.retrieved.get_abs_path(), 'path', 'aiida.out')
        dest_path = 'tmp/{}.out'.format(calculation.id)
        copyfile(src_path, dest_path)
        download_file = ipw.HTML("""<a href="{}" target="_blank"><button>Log file</button></a>""".format(
        dest_path), layout={'width': 'initial'})
        to_display.append(download_file)
    
    if hasattr(calculation.out, 'output_structure') and run_type.value in ['relax', 'vc-relax']:
        global structure_global
        structure_global = calculation.out.output_structure
        structure_ase = structure_global.get_ase()
        viewer = nglview.NGLWidget()
        viewer.add_component(nglview.ASEStructure(structure_ase))  # adds ball+stick
        viewer.add_unitcell()
        formats_to_write = ['xyz', 'cif']
        for fmt in formats_to_write:
            fname = "tmp/{}.{}".format(calculation.id, fmt)
            structure_ase.write(fname)
            download_file = ipw.HTML("""<a href="{}" target="_blank"><button>Output Structure ({})</button></a>""".format(fname, fmt), layout={'width': 'initial'})
            to_display.append(download_file)
        display(viewer)

    if hasattr(calculation.out, 'output_bands') and run_type.value == 'bands':
        bands = calculation.out.output_bands
        from aiida.workflows.user.epfl_theos.quantumespresso.helpers import find_optical_band_gap
        _, homo, lumo, band_gap = find_optical_band_gap(bands,
                            bands.inp.output_band.out.output_parameters.get_dict()['number_of_electrons'],
                            also_homo_lumo_bandgap=True)
        fermi = bands.inp.output_band.out.output_parameters.get_dict()['fermi_energy']
        if homo>lumo:
            homo = fermi
            lumo = fermi
        with out_results:
            display(ipw.HTML("<strong>Electronic bands</strong>: (uuid: {}, pk: {})".format(bands.uuid,bands.pk)))
            bands.show_mpl(bands_linewidth=1.5,y_min_lim=homo-fermi-10.,y_max_lim=lumo-fermi+10.,y_origin=fermi,
                           prettify_format='latex_simple')
            display(ipw.HTML("<strong>Band gap</strong> [eV]: {}".format(band_gap)))
    if to_display:
        to_display.insert(0,ipw.HTML("Download output file(s): "))
        display(ipw.HBox(to_display))
    calculation_settings()

In [ ]:
submit_out = ipw.Output()
structure_widget = StructureUploadWidget()
with submit_out:
    run_btn.on_click(submit_calculation)
    calculation_settings()
display(structure_widget, submit_out)

In [ ]:
#path.join(calculation.out.retrieved.get_abs_path(), 'path', 'aiida.out')

In [ ]:
setup_calc()

In [ ]:
# if 'output_band' in calculation and run_type.value == 'scf':
#     bands = calculation['output_band']
#     from common.helpers.helpers import find_optical_band_gap
#     _, homo, lumo, band_gap = find_optical_band_gap(bands,
#                         bands.inp.output_band.out.output_parameters.get_dict()['number_of_electrons'],
#                         also_homo_lumo_bandgap=True)
#     fermi = bands.inp.output_band.out.output_parameters.get_dict()['fermi_energy']
#     if homo>lumo:
#         homo = fermi
#         lumo = fermi
#     with out_results:
#         display(ipw.HTML("<strong>Electronic bands</strong>: (uuid: {}, pk: {})".format(bands.uuid,bands.pk)))
#         bands.show_mpl(bands_linewidth=1.5,y_min_lim=homo-fermi-10.,y_max_lim=lumo-fermi+10.,y_origin=fermi,
#                        prettify_format='latex_simple')
#         display(ipw.HTML("<strong>Band gap</strong> [eV]: {}".format(band_gap)))

In [ ]:
# import bqplot as bq
# import numpy as np
# from bqplot import Figure

# # x_max = np.pi / c
# x_data = np.linspace(0.0, 1, 65)
# y_datas = calculation.out.output_band.get_bands().transpose()
# y_datas.shape
# lines = bq.Lines(x=x_data, y=y_datas, scales={'x': bq.LinearScale(), 'y': bq.LinearScale()})
# Figure(marks=[lines])